<a href="https://colab.research.google.com/github/Shrouk-Adel/Fine_Tuning_Qwen2.5_LLaMA_Factory/blob/main/LLM_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Setup

In [ ]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb
!pip install json-repair
!pip install -qU faker==35.2.0
!pip install -qU vllm==0.7.2

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory & pip install -e ".[torch,metrics]"

In [ ]:
from google.colab import userdata
import wandb

# Get the wandb API key from userdata
wandb_api_key = userdata.get('wandb')

# Pass the API key to the `key` parameter
wandb.login(key=wandb_api_key)

Hf_token = userdata.get('HF_Token')
!huggingface-cli login --token {Hf_token}

## Imports

In [ ]:
import json
import os
from os.path import join
from tqdm.auto import tqdm
import random
import requests

from pydantic import BaseModel,Field
from typing import List,Literal,Optional
from datetime import datetime


import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch

data_dir ='/gdrive/MyDrive/Fine-Tuning'
base_model_Id ='Qwen/Qwen2.5-1.5B-Instruct'

device='cuda'
torch_dtype=None


def parse_json(text):
  try:
    return json_repair.loads(text)
  except:
    return None

## Tasks

In [ ]:
story = """
ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
 حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
 تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

 الأبعاد الثلاثة للعلاقة بالمال
بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
 البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
 النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
 أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

الاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته بقدرته على توفير
المتعة والراحة. ومع ذلك، قد يصبح
البعض مدمنا على الإنفاق، في حين يتجه آخرون إلى التقشف المفرط خوفا من المستقبل.

الإدارة (M): أصحاب هذا النمط يعتبرون المال مسؤولية تتطلب التخطيط الدقيق. لكن في بعض الحالات،
 قد يتحول الأمر إلى هوس مفرط بإدارة الإنفاق، مما يؤثر سلبا على العلاقات الشخصية.

 كيف تؤثر العائلة على علاقتنا بالمال؟
يشير التقرير إلى أن التجارب الأسرية تلعب دورا رئيسيا في تحديد
 "الشخصية المالية" لكل فرد، على سبيل المثال، إذا كان أحد الوالدين يعتمد على المال
كمكافأة للسلوك الجيد، فقد يتبنى الطفل لاحقا النمط نفسه في حياته البالغة.

لتحليل هذه التأثيرات بشكل دقيق، طورت رابطة العلاج المالي
(Financial Therapy Association) أداة تسمى مخطط الجينوم المالي (Money Genogram)،
وهو نموذج يُستخدم لتحديد الأنماط المالية داخل العائلة.

تتضمن هذه الأداة:

رسم شجرة عائلية.
تصنيف أفراد العائلة وفقا للأبعاد الثلاثة للعلاقة بالمال (A ،U ،M).
تحديد ما إذا كان السلوك المالي لكل فرد صحيا (+) أو غير صحي (-).
على سبيل المثال، إذا نشأ شخص في عائلة
اعتادت على الإنفاق المفرط، فقد يكون لديه ميل قوي إلى اتباع النمط نفسه،
 أو العكس تماما، حيث يصبح مقتصدا بشكل مبالغ فيه كرد فعل نفسي.
"""

### Details Extraction

In [ ]:
# build schema for sturctured output
# {
#     'stroy_title':' ',
#     'story_keywords':['kw1','kw2',....],
#     'story_summary':[' ',,,,,,,' '],
#     'story_category':[]
#     'story_entity':{
#            'entity_value':'القاهرة',
#            'entity_type':'place'
#             }
# }

Entity_type = Literal['person-male','person-female','place','money','product','location','envent','time','law','quantity'
                       'artifact','organization','disease']

Categories =Literal['Politics','Arts','Health','Technology','Sports','Economy','Science','Entertainment','not-specified']

class Entity(BaseModel):
  entity_value:str =Field(...,Description='the actual name or value of the entity')
  entity_type:Entity_type =Field(...,Description='Type of recognized entity')

class NewsDetails(BaseModel):
  story_title:str =Field(...,min_length=5,max_length=300,
                         Description='A fully informative and SEO Optimize title for the story')

  story_keywords :List[str] =Field(...,min_items=1,Description ='Relevant keywords related to story')

  story_summary :List[str] =Field(...,min_items=1,max_items = 5,Description ='summarized key points about story points (1-5)')

  story_category :Categories =Field(..., Description ='Category of the news story')

  story_entity:List[Entity]=Field(...,min_items=1 , max_items=10 , Description ='List of identified entities in the story')



In [ ]:
Details_extraction_messages=[
    {
        'role':'system',
        'content':'\n'.join([
            'you are NLP data parser',
            'generate the output in the same story language',
            'Extract details as mentioned as text',
            'Do not generate any introduction or conclusion',
            'you have to extract json details from text according to the pydantic details',
        ])
    },{
        'role':'user',
        'content':'\n'.join([
            '## Story:',
            story.strip(),
            '',

            'the pydantic details:',
            json.dumps(
                NewsDetails.model_json_schema(),ensure_ascii=False
                )
        ])
    }
]

### Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_Id,
    device_map='auto',
    torch_dtype=torch_dtype

)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

In [ ]:
def generate_response(message):
  text = tokenizer.apply_chat_template(
      message,
      tokenize=False,
      add_generation_prompt=True
  )

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

response =generate_response(Details_extraction_messages)

In [ ]:
print(response)

{
  "story_title": "The Role of Family in Financial Relationships",
  "story_keywords": [
    "financial relationships",
    "family influence",
    "moneymaking",
    "wealth management",
    "child development"
  ],
  "story_summary": [
    "Forbes reported that family plays a crucial role in shaping individuals' financial relationships.",
    "The relationship between individuals and money is influenced by inherited behavioral patterns across generations."
  ],
  "story_category": "Economy",
  "story_entity": [
    {
      "$ref": "#/$defs/Entity",
      "entity_value": "Forbes",
      "entity_type": "Organization"
    },
    {
      "$ref": "#/$defs/Entity",
      "entity_value": "Shain Entine",
      "entity_type": "Person-Female"
    },
    {
      "$ref": "#/$defs/Entity",
      "entity_value": "Financial Therapy Association",
      "entity_type": "Organization"
    }
  ]
}


### Translation

In [ ]:
# {
#     'translation_title':' ',
#     'translation_content':''
# }

class TranslatedStory(BaseModel):
  translation_titl:str =Field(...,min_length =5,Description ='genrate title for the translated story')
  translation_content:str =Field(...,min_lenght =5,Desciption ='Tranlated Content for the new Story ')


target_language='English'
translation_messages=[
    {

        'role':'system',
        'content':'\n'.join([
            'you are a professional translator',
            'translate story for the target language',
            'do not generate any introduction or conclusion',
            'follow the provided schema to generate json'
        ])
    },
    {
        'role':'user',
        'content':'\n'.join([
            '## Story:',
            story.strip(),
            '',

            'the pydantic details:',
            json.dumps(
                TranslatedStory.model_json_schema(),ensure_ascii=False
                ),
            '',

            'Target language:',
             target_language,
        ])
    }
]


In [ ]:
response =generate_response(translation_messages)

In [ ]:
print(response)

```json
{
  "translation_titl": "Family Influence on Financial Relationships",
  "translation_content": "Forbes magazine reported that the family plays a crucial role in shaping individuals' relationships with money, influenced by inherited financial behaviors across generations."
}
```


In [ ]:
json_repair.loads(response)

{'translation_titl': 'Family Influence on Financial Relationships',
 'translation_content': "Forbes magazine reported that the family plays a crucial role in shaping individuals' relationships with money, influenced by inherited financial behaviors across generations."}

## Evaluation DeepSeek-r1

In [ ]:
from google.colab import userdata
from openai import OpenAI

OpenRouter_api_key =userdata.get('open_router').strip()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OpenRouter_api_key,
)

completion = client.chat.completions.create(

  model="deepseek/deepseek-r1:free",
  messages= Details_extraction_messages,
  temperature=0.2,
)
print(completion.choices[0].message.content)

```json
{
  "story_title": "دور العائلة في تشكيل العلاقة بالمال: الأبعاد الثلاثة وتأثيرها على الشخصية المالية",
  "story_keywords": [
    "العائلة",
    "الشخصية المالية",
    "فوربس",
    "الأبعاد المالية",
    "رابطة العلاج المالي",
    "مخطط الجينوم المالي"
  ],
  "story_summary": [
    "تأثير العائلة على أنماط السلوك المالي عبر الأجيال وفقًا لدراسات شاين إنيت",
    "ثلاثة أبعاد للعلاقة بالمال: الاكتساب (A)، الاستخدام (U)، الإدارة (M)",
    "ارتباط السلوك المالي بتجارب الطفولة مثل استخدام المال كمكافأة أو الإنفاق المفرط",
    "دور أداة مخطط الجينوم المالي في تحليل الأنماط المالية العائلية",
    "إمكانية تحوّل الأنماط المالية إلى سلوكيات غير صحية كالهوس بالثروة أو التقشف المفرط"
  ],
  "story_category": "Economy",
  "story_entity": [
    {
      "entity_value": "فوربس",
      "entity_type": "organization"
    },
    {
      "entity_value": "شاين إنيت",
      "entity_type": "person-male"
    },
    {
      "entity_value": "رابطة العلاج المالي",
      "entity_type": "organization"
    

In [ ]:
completion_t = client.chat.completions.create(

  model="deepseek/deepseek-r1:free",
  messages= translation_messages,
  temperature=0.2,
)
print(completion_t.choices[0].message.content)

```json
{
  "translation_titl": "Forbes Reports: How Family Shapes Our Financial Relationships and Behaviors",
  "translation_content": "Forbes magazine highlighted the pivotal role families play in shaping individuals’ relationships with money, noting that these relationships are influenced by financial behavior patterns inherited across generations.\n\nThe report, based on research by Professor Shane Enitt on financial well-being, explains that everyone has a \"financial personality\" determined by their interaction with money. This personality is directly influenced by family upbringing and childhood experiences.\n\n**The Three Dimensions of Financial Relationships**\nAccording to the study, three key dimensions shape our relationship with money:\n\n1. **Acquisition (A):** Individuals in this category view money as a collectible commodity, seeing wealth accumulation as an end goal. The downside includes risks of developing an obsession with wealth or, conversely, outright rejection 

In [ ]:
result=completion.choices[0].message.content
type(result)

str

In [ ]:
json_repair.loads(result)

{'story_title': 'دور العائلة في تشكيل العلاقة بالمال: الأبعاد الثلاثة وتأثيرها على الشخصية المالية',
 'story_keywords': ['العائلة',
  'الشخصية المالية',
  'فوربس',
  'الأبعاد المالية',
  'رابطة العلاج المالي',
  'مخطط الجينوم المالي'],
 'story_summary': ['تأثير العائلة على أنماط السلوك المالي عبر الأجيال وفقًا لدراسات شاين إنيت',
  'ثلاثة أبعاد للعلاقة بالمال: الاكتساب (A)، الاستخدام (U)، الإدارة (M)',
  'ارتباط السلوك المالي بتجارب الطفولة مثل استخدام المال كمكافأة أو الإنفاق المفرط',
  'دور أداة مخطط الجينوم المالي في تحليل الأنماط المالية العائلية',
  'إمكانية تحوّل الأنماط المالية إلى سلوكيات غير صحية كالهوس بالثروة أو التقشف المفرط'],
 'story_category': 'Economy',
 'story_entity': [{'entity_value': 'فوربس', 'entity_type': 'organization'},
  {'entity_value': 'شاين إنيت', 'entity_type': 'person-male'},
  {'entity_value': 'رابطة العلاج المالي', 'entity_type': 'organization'},
  {'entity_value': 'مخطط الجينوم المالي', 'entity_type': 'quantityartifact'}]}

### knowledge Distillation

In [ ]:
raw_data_paths =join(data_dir,'Datasets','news-sample.jsonl')

raw_data =[]
for line in open(raw_data_paths):
  if line.strip()=='':
    continue

  raw_data.append(json.loads(line.strip()))

# shuffle data
random.Random(101).shuffle(raw_data)
print(f'Raw Data: {len(raw_data)}')

Raw Data: 2400


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
print(raw_data[0]['content'])

يواصل المعهد العربي في باريس استقبال زواره في معرض ما تقدمه فلسطين للعالم لإطلاعهم على الإرث الثقافي والفني للفلسطينيين؛ من خلال أعمال فنية لآمالهم وصور لواقعهم الأليم تحت الاحتلال. 
 ويرى رئيس المعهد جاك لانغ -الذي أُعيد انتخابه قبل أيام للدورة الرابعة- ما يحدث في غزة حاليا جراء العدوان الإسرائيلي أنه كارثة. 
 والمعهد هو مركز ثقافي وواجهة دبلوماسية يديرها لانغ منذ 2013 ويقع على ضفة نهر السين في باريس. 
 وأشار لانغ، الذي شغل سابقا منصب وزير الثقافة بفرنسا، إلى أن المعرض هو إهداء للشعب الفلسطيني، ومُدّد ليستقبل مزيدا من الزوار حتى 31 ديسمبركانون الأول الجاري. 
 ويضم المعرض، الذي افتُتح أواخر مايوأيار الماضي، حسب لانغ العديد من المعارض الفرعية عن فلسطين وعن غزة بالتحديد، من بينها معرض الصور اليومية عن الحياة في غزة. 
 كما يشتمل على معرض الصور الفوتوكرومية القائم على تلوين صور من فلسطين تعود للقرن الـ19. 
 ويعرض الفنان الفلسطيني محمد أبو سل عملا فريدا بعنوان مترو غزة، وهو عبارة عن عمل تركيبي متعدد الوسائط، لاقى إعجابا من الزوار. 
 ويحضر الشاعر الفلسطيني الراحل محمود درويش من خلال أشعاره ف

In [ ]:
import time

cloud_model_Id="deepseek/deepseek-r1:free"

price_per_1m_input_tokens = 0.29
price_per_1m_output_tokens = 0.39


prompt_tokens =0 # calculate input tokens
completion_tokesn=0 # calculate consumption tokens
ix=0

save_to=join(data_dir,'Datasets','sft.jsonl')

for story in tqdm(raw_data):
    sample_Details_extraction_messages=[
        {
            'role':'system',
            'content':'\n'.join([
                'you are NLP data parser',
                'generate the output in the same story language',
                'Extract details as mentioned as text',
                'Do not generate any introduction or conclusion',
                'you have to extract json details from text according to the pydantic details'
            ])
        },{
            'role':'user',
            'content':'\n'.join([
                '## Story:',
                story['content'].strip(),
                '',

                'the pydantic details:',
                json.dumps(
                    NewsDetails.model_json_schema(),ensure_ascii=False
                    ),
                '',

                'Story Details:',
                '```json'
            ])
        }
    ]

    response = client.chat.completions.create(
      model=cloud_model_Id,
      messages=sample_Details_extraction_messages,
      temperature=0.2,

      )

    if response.choices[0].finish_reason !='stop':
      prompt_tokens += response.usage.prompt_tokens
      continue

    llm_response= response.choices[0].message.content
    llm_resp_dic =parse_json(llm_response)

    if llm_resp_dic is None:
      continue

    with open(save_to,'a',encoding='utf-8') as dest:
      dest.write(json.dumps({
            "id": ix,
            "story": story['content'].strip(),
            "task": "Extrat the story details into a JSON.",
            "output_scheme": json.dumps( NewsDetails.model_json_schema(), ensure_ascii=False ),
            "response": llm_resp_dic,
        }, ensure_ascii=False, default=str)  + "\n" )

    ix+=1
    prompt_tokens +=response.usage.prompt_tokens
    completion_tokesn +=response.usage.completion_tokens


    if (ix % 10) == 0:
      cost_input =(prompt_tokens / 1_000_000) * price_per_1m_input_tokens
      cost_output =(completion_tokesn / 1_000_000) * price_per_1m_output_tokens
      total_cost = cost_input + cost_output
      print(f"Processed {ix} samples, Cost: ${total_cost:.4f}")


In [ ]:
save_to = join(data_dir, "datasets", "sft.jsonl")

ix = 0
for story in tqdm(raw_data):

    for targeted_lang in ["English", "French"]:
        sample_translation_messages = [
            {
                "role": "system",
                "content": "\n".join([
                    "You are a professional translator.",
                    "You will be provided by an Arabic text.",
                    "You have to translate the text into the `Targeted Language`.",
                    "Follow the provided Scheme to generate a JSON",
                    "Do not generate any introduction or conclusion."
                ])
            },
            {
                "role": "user",
                "content": "\n".join([
                    "## Pydantic Details:",
                    json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
                    "",

                    "## Targeted Language or Dialect:",
                    targeted_lang,
                    "",

                    "## Story:",
                    story['content'].strip(),
                    "",

                    "## Translated Story:",
                    "```json"
                ])
            }
        ]

        response = client.chat.completions.create(
        model=cloud_model_Id,
        messages=sample_translation_messages,
        temperature=0.2,

            )

        if response.choices[0].finish_reason != "stop":
            prompt_tokens += response.usage.prompt_tokens
            continue

        llm_response = response.choices[0].message.content
        llm_resp_dict = parse_json(llm_response)

        if not llm_resp_dict:
            continue

        with open(save_to, "a", encoding="utf8") as dest:
            dest.write(json.dumps({
                "id": ix,
                "story": story['content'].strip(),

                "output_scheme": json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
                "task": f"You have to translate the story content into {targeted_lang} associated with a title into a JSON.",

                "response": llm_resp_dict,
            }, ensure_ascii=False, default=str)  + "\n" )

        ix += 1
        prompt_tokens += response.usage.prompt_tokens
        completion_tokens += response.usage.completion_tokens

        if(ix % 3) == 0:
            cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
            cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
            total_cost = cost_input + cost_output

            print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")

## Formate Fine-tuning Dataset

In [ ]:
sft_data_path =join(data_dir,'Datasets','sft.jsonl')
llm_finetuning_data=[]

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

for line in open(sft_data_path):
  if line.strip() == "":
    continue

  rec =json.loads(line.strip())

  llm_finetuning_data.append({
      'system':system_message,
      'instruction':'\n'.join([
          '# story',
          rec['story'],

          '# task',
          rec['task'],

          '#output_schema',
          rec['output_scheme'],

          '# output json',
          "```json"

          ''
      ]),
      'input':'',
      'output':'\n'.join([
          '```json',
          json.dumps(rec['response'],ensure_ascii=False ,default=str),
          "```"
      ]),
      'history':[]
    })


random.Random(101).shuffle(llm_finetuning_data)

In [ ]:
len(llm_finetuning_data)

2766

In [ ]:
train_sample_sz= 2700

train_ds =llm_finetuning_data[:train_sample_sz]
valid_ds=llm_finetuning_data[train_sample_sz:]

# create new folder
os.makedirs(join(data_dir,'Datasets','LLaMaFactory-Finetuning-data'),exist_ok=True)

with open(join(data_dir,'Datasets','LLaMaFactory-Finetuning-data','train.json'),'w') as dest:
  json.dump(train_ds,dest,ensure_ascii=False,default=str)

with open(join(data_dir,'Datasets','LLaMaFactory-Finetuning-data','valid.json'),'w',encoding='utf8') as dest:
  json.dump(valid_ds,dest,ensure_ascii=False,default=str)


In [ ]:
join(data_dir, "datasets", "llamafactory-finetune-data", "val.json")

'/gdrive/MyDrive/Fine-Tuning/datasets/llamafactory-finetune-data/val.json'

In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
# ```
   "news_finetune_train": {
        "file_name": "/gdrive/MyDrive/Fine-Tuning/Datasets/LLaMaFactory-Finetuning-data/train.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    },
    "news_finetune_val": {
        "file_name": "/gdrive/MyDrive/Fine-Tuning/Datasets/LLaMaFactory-Finetuning-data/valid.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
# ```

# https://wandb.ai/mr-bakrianoo/llamafactory/runs/apwbkni9
# https://wandb.ai/mr-bakrianoo/llamafactory/runs/c5tf0q90

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml
### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
template: qwen
cutoff_len: 3500
max_samples: 550
overwrite_cache: true
preprocessing_num_workers: 16
dataloader_num_workers: 4

### output
resume_from_checkpoint: /gdrive/MyDrive/Fine-Tuning/Datasets/LLaMaFactory-Finetuning-data/models/checkpoint-500
output_dir: /gdrive/MyDrive/Fine-Tuning/Datasets/LLaMaFactory-Finetuning-data/models
logging_steps: 10
save_steps: 50
plot_loss: true
# overwrite_output_dir: true
save_only_model: false

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 8
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1 # don not change learning rate for first 10 % from data
bf16: true # store half of weights in memory
ddp_timeout: 180000000
resume_from_checkpoint: null

### eval
eval_dataset: news_finetune_val
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100


report_to: wandb
run_name: news_finetuning_llamafactory

# push_to_hub: true
# export_hub_model_id : "shroukAdel/news_analyzer"
# hub_private_repo: true
# hub_strategy: checkpoint # update for every checkpoint


Overwriting /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!cd LLaMA-Factory && pip install -e .


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

2025-05-19 09:30:15.126343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747647015.321576    5359 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747647015.377313    5359 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 09:30:15.781042: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[INFO|2025-05-19 09:30:29] llamafactory.hparams.parser:143 >> Resuming training from /gdrive/MyDrive/Fine-Tuning/Datasets/LLa

### Evaluation

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_Id,
    device_map='auto',
    torch_dtype=torch_dtype

)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

In [ ]:
finetuned_model_id = "/gdrive/MyDrive/Fine-Tuning/Datasets/LLaMaFactory-Finetuning-data/models"
model.load_adapter(finetuned_model_id)

In [ ]:
response =generate_response(Details_extraction_messages)

In [ ]:
parse_json(response)

{'story_title': 'تأثير العائلة في علاقات الأفراد بالمال',
 'story_keywords': ['العائلة',
  'علاقات المال',
  'التاريخ المالي',
  'الأصول',
  'التربية'],
 'story_summary': ['مجلة فوربس تشير إلى أهمية العائلة في تشكيل العلاقة بالمال.',
  'العلاقات المالية تتأثر بمختلف النماذج الماليية مثل الاستكثار، استخدام المال، وإدارة المال.',
  'التجارب الأسرية تحدد شخصيات مالية مختلفة لدى الأفراد.',
  'الدراسة تقدم أدوات مثل مخطط الجينوم المالي لتقييم التأثيرات السلبية.',
  'العائلات التي تعاني من نمط استهلاك مالي مفرط قد تواجه مشكلات نفسية.'],
 'story_category': 'Economy',
 'story_entity': [{'entity_value': 'فوربس', 'entity_type': 'organization'},
  {'entity_value': 'شاين إنيت', 'entity_type': 'person-male'},
  {'entity_value': 'رابطة العلاج المالي', 'entity_type': 'organization'},
  {'entity_value': 'Money Genogram', 'entity_type': 'product'}]}

In [ ]:
response=generate_response(translation_messages)

In [ ]:
parse_json(response)

{'translation_titl': 'The Role of Family in Financial Relationships',
 'translation_content': 'Forbes magazine reported that family plays a central role in shaping individuals\' financial relationships, influenced by inherited financial behaviors across generations.\n\nThis report is based on research conducted by Professor Shane Einthoven on wealth management, explaining that each person has an individual finance profile determined by their interaction with money, which is directly affected by family upbringing and childhood experiences.\n\nThe three dimensions of our relationship with money\nAccording to the study, there are three main dimensions that form our financial relationship:\n\nAcquisition (A): Individuals belonging to this dimension tend to view money as a commodity to be accumulated, considering wealth accumulation as a goal itself. The negative aspect of this behavior is the potential for it to lead to a love affair with wealth or vice versa, i.e., complete rejection of w

#### Cost Estimation for the model after fintuned
- Test model speed on generation

In [ ]:
from faker import Faker
fake=Faker('ar')
fake.text(max_nb_chars=random.randint(150,200))


'وباستثناء إختار سابق بكلا. يرتبط أسابيع يبق الأراضي. فشكّل السيء معاملة. تونس دارت لبولندا مشروط كرسي الآخر.'

In [ ]:
from faker import Faker
from  datetime import datetime
fake=Faker('ar')

input_tokens= 0
output_tokens =0
start_time =datetime.now()

for i in tqdm(range(30)):
  prompt = fake.text(max_nb_chars=random.randint(150,200))

  messages=[
  {
      'role':'user',
      'content':prompt
  }
  ]

  response =generate_response(messages)

  input_tokens +=len(tokenizer.apply_chat_template(messages))
  output_tokens +=len(tokenizer.encode(response))


total_time =(datetime.now()-start_time).total_seconds()

print(f"Total Time:{total_time} seconds")
print(f"Input Tokens:{input_tokens}")
print(f"Output Tokens:{output_tokens}")
print(f"Total Tokens:{input_tokens+output_tokens}")

  0%|          | 0/30 [00:00<?, ?it/s]

Total Time:613.942767 seconds
Input Tokens:2395
Output Tokens:11245
Total Tokens:13640


In [ ]:
13640 /613  # generate 22 token per second

22.251223491027734

## vLLM

In [ ]:
base_model_Id ='Qwen/Qwen2.5-1.5B-Instruct'
lora_model_Id ="/gdrive/MyDrive/Fine-Tuning/Datasets/LLaMaFactory-Finetuning-data/models"

!nohup vllm serve "{base_model_Id}" --dtype=half --gpu-memory-utilization 0.8 --max-lora-rank 64 --enable-lora --lora-modules news-lora="{lora_model_Id}" &

In [ ]:
!tail -n 30 nohup.out

tail: cannot open 'nohup.out' for reading: No such file or directory


### Inference

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
prompt =tokenizer.apply_chat_template(translation_messages,
                                      tokenize=False,
                                      add_generation_prompt=True)
prompt

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'<|im_start|>system\nyou are a professional translator\ntranslate story for the target language\ndo not generate any introduction or conclusion\nfollow the provided schema to generate json<|im_end|>\n<|im_start|>user\n## Story:\nذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،\n حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.\n\nالتقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول\nالرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة\n تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.\n\n الأبعاد الثلاثة للعلاقة بالمال\nبحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:\n\nالاكتساب (A): يميل الأفراد الذين ينتمون لهذا\n البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون\nفي تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا\n النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،\n أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.\n\nالاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته ب

In [ ]:
vllm_model_id='news-lora'

llm_response=requests.post("http://localhost:8000/v1/completions",json={
    'model':vllm_model_id,
    'prompt':prompt,
    'max_tokens':1000,
    'temperature':0.3
})

llm_response.text

'{"id":"cmpl-bae13276e7974f1e956111353610ca80","object":"text_completion","created":1747832275,"model":"news-lora","choices":[{"index":0,"text":"```json\\n{\\"translation_titl\\": \\"The Role of Family in Financial Relationships\\", \\"translation_content\\": \\"Forbes magazine reported that the family plays a crucial role in shaping individuals\' financial relationships, as these relationships are influenced by inherited financial behaviors across generations.\\\\n\\\\nThe report, based on research by Professor Shane Enye on financial well-being, explains that each person has a \\"financial personality\\" determined by their interaction with money, which is directly affected by family upbringing and childhood experiences.\\\\n\\\\nThe three dimensions of the financial relationship\\\\nAccording to the study, there are three main dimensions that form our financial relationship:\\\\n\\\\nA: Individuals belonging to this dimension tend to view money as a commodity to be accumulated, seei

In [ ]:
llm_response.json()

{'id': 'cmpl-bae13276e7974f1e956111353610ca80',
 'object': 'text_completion',
 'created': 1747832275,
 'model': 'news-lora',
 'choices': [{'index': 0,
   'text': '```json\n{"translation_titl": "The Role of Family in Financial Relationships", "translation_content": "Forbes magazine reported that the family plays a crucial role in shaping individuals\' financial relationships, as these relationships are influenced by inherited financial behaviors across generations.\\n\\nThe report, based on research by Professor Shane Enye on financial well-being, explains that each person has a "financial personality" determined by their interaction with money, which is directly affected by family upbringing and childhood experiences.\\n\\nThe three dimensions of the financial relationship\\nAccording to the study, there are three main dimensions that form our financial relationship:\\n\\nA: Individuals belonging to this dimension tend to view money as a commodity to be accumulated, seeing wealth accum

### Load Testing

In [ ]:
%%writefile locust.py
import random
import json
from locust import HttpUser,task,between,constant
from transformers import AutoTokenizer
from faker import Faker

faker=Faker('ar')

class completionLoadTest(HttpUser):
  wait_time =between(1,3)

  @task
  def post_completeion(self):
    model_id='news-lora'
    prompt =faker.text(max_nb_chars=random.randint(150,200))

    message={'model':model_id,
    'prompt':prompt,
    'max_tokens':512,
    'temperature':0.3
    }

    llm_response=self.client.post("/v1/completions",json= message)

    if llm_response.status_code ==200:
      with open('./vllm_token_.text','a') as dest:
        dest.write(json.dumps({
            "prompt":prompt,
            'resposne':llm_response.json()["choices"][0]["text"]
        },ensure_ascii=False )+"\n")

Overwriting locust.py


In [ ]:
!locust --headless -f locust.py --host=http://localhost:8000 -u 20 -r 1 -t "60s" --html=locust_result.html

[2025-05-21 14:01:57,709] befdd81360ad/INFO/locust.main: Starting Locust 2.37.4
[2025-05-21 14:01:57,726] befdd81360ad/INFO/locust.main: Run time limit set to 60 seconds
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2025-05-21 14:01:57,756] befdd81360ad/INFO/locust.runners: Ramping to 20 users at a rate of 1.00 per second
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
POST     /v1/completions       1     0(0.00%) |    801     801     801    801 |    0.00        0.00
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
       